In [1]:
#import data processing libraries

import os
import pandas as pd
import numpy as np
import math as math
import datetime as dt
from scipy import stats

#db connection libraries
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras

In [2]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

#suppress scientific notation to 2 decimal places

pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [3]:
df = pd.read_parquet('6.2_FM_bloom_saf_scored_limits_approved (1).parquet')

df.head()

,previous_is_iprs_validated,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,mobile_number,is_location_mapped,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,any_bloom2_1day,total_outstanding_sum,safaricom_loan_balance_sum,loan_balance,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,repayments_by_dd_vs_principal_mean,due_date_rm_ge_rm_add_back_old,due_date_rm_ge_rm_1d_old,due_date_100_check,repayments_by_erd_vs_principal_mean,rllvr_date_rm_ge_rm_add_back_old,rllvr_date_102_check,never_borrowed_flag_old,repayments_by_dd_vs_total_expected_repayment_by_dd_mean,due_date_rm_ge_rm_add_back_new,hurdle_rate_by_due_date_mean,hurdle_rate_by_end_rollover_date_mean,hurdle_rate_by_dpd30_mean,repayments_by_erd_vs_total_expected_repayment_by_erd_mean,rllvr_date_rm_ge_rm_add_back_new,rllvr_date_rm_ge_rm_1d_new,never_borrowed_flag_new,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,due_date_rm_ge_rm_1d,never_borrowed_flag,lrr_update_flag,reinstatement_reason_x,rein_7_limit,idm_recommendation,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,l21_day_graduation_flag_new_limits,21_day_graduation_flag,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit,updated_new_7_limit,updated_new_1_limit,zeroize_final_1_limit,ftd_update_flag,reinstatement_reason_y,model_630_max_global_limit,rein_update_flag,reinstatement_reason,total_max_global_limit,hurdle_rate_dd,hurdle_rate_erd,hurdle_rate_dpd30,rllvr_date_rm_ge_rm_never_borrowed,consolidated_repayments_by_dd,consolidated_repayments_by_erd,consolidated_repayments_by_dpd30,max_global_limit,previous_max_global_limit,max_limit_diff,model_version,created_at,rules_summary_narration,limit_reason,record_added_to_warehouse_on_timestamp
0,None,7356130,2023-05-14,2023-06-12,30,1355574,30,1,relax_rules,0,Yes,1,24782075,True,254724546505,True,3000,254724546505,1,600,7,52000,52000,2023-03-29,2023-04-05,2023-04-04,2023-04-05,2023-04-08,2023-07-07,-1,2,closed_early_repayment,461,52000,2023-03-29,1,1,75,1,-1,1,1,7,1,1,52000,52000,0,52000,-4,False,0,0,0,0,2423200,4675500,0,200000,200000,0,151900,151900,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,None,0,Approve,1,0,0,0,1,1,1,Band 7,Band 2,0,0,0,1,1,1,1,0,0,0,338894,203336,203336,338894,203336,203336,200000,200000,0,153000,pass,pass,200000,200000,0,200000,200000,0,0,200000,200000,200000,200000,0,0,1,1,200000,

In [4]:
df_loans = pd.read_csv('loans_fact_table_materialized_summary_view_202306161416.csv')

df_loans['store_number'] = df_loans['store_number'].astype(str)

df_loans.head()

,loan_mifos_id,store_number,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
0,36185,7040709,11000,2022-03-03,2022-03-10,12397,0,12397,NaN,433,2
1,262987,7622426,300,2022-12-06,2022-12-07,333,301,32,NaN,NaN,2
2,29932,505195,20700,2022-02-18,2022-02-25,27777,0,27777,NaN,423,2
3,88604,620511,200,2022-05-06,2022-05-07,264,5,259,NaN,NaN,2
4,87407,7304804,200,2022-05-06,2022-05-07,268,5,263,NaN,NaN,2


In [5]:
df_loans.shape

(22611, 11)

In [6]:
df_active_loans = df_loans[df_loans['safaricom_loan_balance'] != 0]

df_active_loans.shape

(22326, 11)

In [7]:
df_active_loans.head(3)

,loan_mifos_id,store_number,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
0,36185,7040709,11000,2022-03-03,2022-03-10,12397,0,12397,NaN,433,2
1,262987,7622426,300,2022-12-06,2022-12-07,333,301,32,NaN,NaN,2
2,29932,505195,20700,2022-02-18,2022-02-25,27777,0,27777,NaN,423,2


In [8]:
df_2 = df[['store_number', 'national_id', 'final_21_limit', 'final_7_limit', 'final_1_limit', 'max_global_limit']]

df_2.head()

,store_number,national_id,final_21_limit,final_7_limit,final_1_limit,max_global_limit
0,7356130,24782075,200000,200000,200000,200000
1,7152964,13087340,125100,68300,34200,125100
2,7806607,13337230,48900,28500,25100,48900
3,7627377,22114918,30400,16800,0,30400
4,7650159,25028452,200000,126900,0,200000


In [9]:
df_2 = pd.merge(df_2, df_active_loans, on='store_number', how='left')

In [10]:
df_2['national_id'].nunique()

178773

In [11]:
df_2.shape

(191483, 16)

In [12]:
group = df_2.groupby(by=['national_id'])['store_number'].count()

group.head()

national_id
0006255     1
00121323    1
00126055    1
00331912    1
004277      1
Name: store_number, dtype: int64

In [13]:
value = group[group > 1]

value.sort_values()

national_id
27092983      2
28501909      2
28498282      2
37404463      2
28494217      2
           ... 
34815942     17
34275352     27
0759292      27
29454260     37
None        735
Name: store_number, Length: 10247, dtype: int64

In [14]:
value.shape

(10247,)

In [15]:
sum(value)

22957

In [16]:
df_3 = pd.merge(value.to_frame(), df_2, on='national_id', how='inner')

df_3.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
0,0328232,3,887347,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0328232,3,7164748,110100,60100,0,110100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0328232,3,7178094,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,03709574,4,7139970,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,03709574,4,7928461,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_31 = df_3[['national_id', 'store_number_y']]

In [18]:
df_31.to_excel('df_3.xlsx')

In [19]:
df_3.shape

(22957, 17)

In [20]:
df_3[['final_21_limit', 'final_7_limit', 'final_1_limit', 'max_global_limit']].sum()

final_21_limit     26788900
final_7_limit      69057800
final_1_limit      11863000
max_global_limit   75318900
dtype: float64

In [21]:
df[['final_21_limit', 'final_7_limit', 'final_1_limit', 'max_global_limit']].sum()

final_21_limit     109415900
final_7_limit      291501800
final_1_limit       58203700
max_global_limit   319000800
dtype: float64

In [22]:
df_3['store_number_x'].min()

2

In [23]:
df_3['store_number_x'].max()

735

In [24]:
with_limits = df_3[df_3['max_global_limit'] > 0]

with_limits.shape

(894, 17)

In [25]:
with_limits.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
1,0328232,3,7164748,110100,60100,0,110100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,10033391,2,834417,187100,102100,0,187100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,1006543,2,201561,0,76700,0,76700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,10134016,4,7002703,0,200000,0,200000,313281,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
104,10147213,2,7075322,0,5900,0,5900,312640,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2


In [26]:
with_limits.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
1,0328232,3,7164748,110100,60100,0,110100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,10033391,2,834417,187100,102100,0,187100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,1006543,2,201561,0,76700,0,76700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,10134016,4,7002703,0,200000,0,200000,313281,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
104,10147213,2,7075322,0,5900,0,5900,312640,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2


In [27]:
with_limits['national_id'].nunique()

777

In [28]:
with_limits['store_number_x'].min()

2

In [29]:
with_limits['store_number_x'].max()

10

In [30]:
with_limits[['final_21_limit', 'final_7_limit', 'final_1_limit', 'max_global_limit']].sum()

final_21_limit     26782900
final_7_limit      69054300
final_1_limit      11860500
max_global_limit   75318900
dtype: float64

In [31]:
with_limits.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
1,0328232,3,7164748,110100,60100,0,110100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,10033391,2,834417,187100,102100,0,187100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,1006543,2,201561,0,76700,0,76700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,10134016,4,7002703,0,200000,0,200000,313281,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
104,10147213,2,7075322,0,5900,0,5900,312640,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2


In [32]:
with_limits.shape

(894, 17)

In [33]:
with_limits_and_loans = with_limits[with_limits['principal_disbursed'].notna()]

with_limits_and_loans.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
92,10134016,4,7002703,0,200000,0,200000,313281,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
104,10147213,2,7075322,0,5900,0,5900,312640,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2
167,10359355,2,7215187,0,200000,0,200000,263184,200000,2022-12-08,2022-12-15,273099,122000,151099,151129,483,2
191,10425987,2,7253109,0,129300,0,129300,313444,116900,2023-06-15,2023-06-22,120547,0,120547,120547,462,2
227,10517437,2,7569765,0,58400,0,58400,157211,107000,2022-07-15,2022-08-05,171200,114500,56700,67908,476,2


In [34]:
with_limits_and_loans.shape

(342, 17)

In [35]:
with_limits = with_limits[['national_id', 'store_number_x', 'store_number_y', 'final_21_limit', 'final_7_limit', 'final_1_limit', 'max_global_limit', 'loan_mifos_id']]

In [36]:
# in_two_not_in_one = scoring_results_refresh_two_without_21_limits[~scoring_results_refresh_two_without_21_limits['store_number'].isin(scoring_results_refresh_one_without_21_limits['store_number'].tolist())]

in_with_limits_not_in_with_limits_and_loans = with_limits[~with_limits['national_id'].isin(with_limits_and_loans['national_id'].tolist())]

in_with_limits_not_in_with_limits_and_loans.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
1,0328232,3,7164748,110100,60100,0,110100,NaN
58,10033391,2,834417,187100,102100,0,187100,NaN
66,1006543,2,201561,0,76700,0,76700,NaN
110,10176279,2,544340,0,1700,0,1700,NaN
136,10252966,2,618683,21000,12300,10500,21000,NaN


In [37]:
in_with_limits_not_in_with_limits_and_loans.shape

(530, 8)

In [38]:
multiple_stores_no_current_loans = in_with_limits_not_in_with_limits_and_loans.sort_values(by=['national_id', 'max_global_limit'], ascending=[True, False])

multiple_stores_no_current_loans.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
1,0328232,3,7164748,110100,60100,0,110100,NaN
58,10033391,2,834417,187100,102100,0,187100,NaN
66,1006543,2,201561,0,76700,0,76700,NaN
110,10176279,2,544340,0,1700,0,1700,NaN
136,10252966,2,618683,21000,12300,10500,21000,NaN


In [39]:
multiple_stores_no_current_loans['national_id'].value_counts()

29356092    2
20983793    2
6104311     2
11336526    2
24084185    2
           ..
22782678    1
22703314    1
22694538    1
22595428    1
9949806     1
Name: national_id, Length: 489, dtype: int64

In [40]:
multiple_stores_no_current_loans[multiple_stores_no_current_loans['national_id'] == '29356092']

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
14311,29356092,2,7656021,0,200000,200000,200000,NaN
14310,29356092,2,7904527,0,8100,0,8100,NaN


In [41]:
multiple_stores_no_current_loans_first = multiple_stores_no_current_loans.drop_duplicates(subset='national_id', keep='first')

multiple_stores_no_current_loans_first.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
1,0328232,3,7164748,110100,60100,0,110100,NaN
58,10033391,2,834417,187100,102100,0,187100,NaN
66,1006543,2,201561,0,76700,0,76700,NaN
110,10176279,2,544340,0,1700,0,1700,NaN
136,10252966,2,618683,21000,12300,10500,21000,NaN


In [42]:
multiple_stores_no_current_loans_first.shape

(489, 8)

In [43]:
multiple_stores_no_current_loans_first['national_id'].nunique()

489

In [44]:
multiple_stores_no_current_loans_first[multiple_stores_no_current_loans_first['national_id'] == '29356092']

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
14311,29356092,2,7656021,0,200000,200000,200000,NaN


In [45]:
batch_1_zeroization = multiple_stores_no_current_loans[~multiple_stores_no_current_loans['store_number_y'].isin(multiple_stores_no_current_loans_first['store_number_y'].tolist())]

batch_1_zeroization.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
465,10967552,2,829542,0,42400,0,42400,NaN
473,10977961,2,637895,71100,38800,0,71100,NaN
516,11036834,2,7019742,0,8000,8000,8000,NaN
766,11336526,3,928643,0,20100,20100,20100,NaN
943,11613645,4,673772,0,89200,0,89200,NaN


In [46]:
batch_1_zeroization = batch_1_zeroization[['national_id', 'store_number_y', 'final_21_limit', 'final_7_limit', 'final_1_limit', 'max_global_limit']]

In [47]:
batch_1_zeroization.shape

(41, 6)

In [48]:
batch_1_zeroization['national_id'].nunique()

41

In [49]:
batch_1_zeroization['store_number_y'].nunique()

41

In [50]:
batch_1_zeroization[['final_21_limit', 'final_7_limit', 'final_1_limit', 'max_global_limit']].sum()

final_21_limit      122200
final_7_limit      2121400
final_1_limit       275700
max_global_limit   2177100
dtype: float64

In [51]:
# batch_1_zeroization.to_excel('batch_1_zeroization.xlsx')

In [52]:
with_limits.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
1,0328232,3,7164748,110100,60100,0,110100,NaN
58,10033391,2,834417,187100,102100,0,187100,NaN
66,1006543,2,201561,0,76700,0,76700,NaN
92,10134016,4,7002703,0,200000,0,200000,313281
104,10147213,2,7075322,0,5900,0,5900,312640


In [53]:
with_limits_and_loans.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
92,10134016,4,7002703,0,200000,0,200000,313281,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
104,10147213,2,7075322,0,5900,0,5900,312640,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2
167,10359355,2,7215187,0,200000,0,200000,263184,200000,2022-12-08,2022-12-15,273099,122000,151099,151129,483,2
191,10425987,2,7253109,0,129300,0,129300,313444,116900,2023-06-15,2023-06-22,120547,0,120547,120547,462,2
227,10517437,2,7569765,0,58400,0,58400,157211,107000,2022-07-15,2022-08-05,171200,114500,56700,67908,476,2


In [54]:
group = with_limits_and_loans.groupby(by=['national_id'])['store_number_y'].count()

group.head()

national_id
10134016    1
10147213    1
10359355    1
10425987    1
10517437    1
Name: store_number_y, dtype: int64

In [55]:
value = group[group == 1]

value.sort_values().head()

national_id
10134016    1
27214513    1
27226852    1
27238735    1
27258539    1
Name: store_number_y, dtype: int64

In [56]:
value.shape

(240,)

In [57]:
with_limits_and_one_active_loans = pd.merge(value.to_frame(), with_limits_and_loans, on='national_id', how='left')

with_limits_and_one_active_loans.head()

,national_id,store_number_y_x,store_number_x,store_number_y_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
0,10134016,1,4,7002703,0,200000,0,200000,313281,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
1,10147213,1,2,7075322,0,5900,0,5900,312640,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2
2,10359355,1,2,7215187,0,200000,0,200000,263184,200000,2022-12-08,2022-12-15,273099,122000,151099,151129,483,2
3,10425987,1,2,7253109,0,129300,0,129300,313444,116900,2023-06-15,2023-06-22,120547,0,120547,120547,462,2
4,10517437,1,2,7569765,0,58400,0,58400,157211,107000,2022-07-15,2022-08-05,171200,114500,56700,67908,476,2


In [58]:
with_limits_and_one_active_loans[['principal_disbursed', 'total_outstanding', 'safaricom_loan_balance']].sum()

principal_disbursed      22820490
total_outstanding        22897929
safaricom_loan_balance   16599724
dtype: float64

In [59]:
with_limits_and_one_active_loans.shape

(240, 18)

In [60]:
with_limits.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
1,0328232,3,7164748,110100,60100,0,110100,NaN
58,10033391,2,834417,187100,102100,0,187100,NaN
66,1006543,2,201561,0,76700,0,76700,NaN
92,10134016,4,7002703,0,200000,0,200000,313281
104,10147213,2,7075322,0,5900,0,5900,312640


In [61]:
with_limits_merge = pd.merge(with_limits, with_limits_and_one_active_loans, left_on=['national_id', 'store_number_y', 'loan_mifos_id'], right_on=['national_id', 'store_number_y_y', 'loan_mifos_id'], how='left')

with_limits_merge.head()

,national_id,store_number_x_x,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x,loan_mifos_id,store_number_y_x,store_number_x_y,store_number_y_y,final_21_limit_y,final_7_limit_y,final_1_limit_y,max_global_limit_y,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
0,0328232,3,7164748,110100,60100,0,110100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10033391,2,834417,187100,102100,0,187100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1006543,2,201561,0,76700,0,76700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10134016,4,7002703,0,200000,0,200000,313281,1,4,7002703,0,200000,0,200000,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
4,10147213,2,7075322,0,5900,0,5900,312640,1,2,7075322,0,5900,0,5900,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2


In [62]:
with_limits_and_one_active_loans_history = with_limits_merge[with_limits_merge.national_id.isin(with_limits_and_one_active_loans.national_id)]

with_limits_and_one_active_loans_history.head()

,national_id,store_number_x_x,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x,loan_mifos_id,store_number_y_x,store_number_x_y,store_number_y_y,final_21_limit_y,final_7_limit_y,final_1_limit_y,max_global_limit_y,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
3,10134016,4,7002703,0,200000,0,200000,313281,1,4,7002703,0,200000,0,200000,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
4,10147213,2,7075322,0,5900,0,5900,312640,1,2,7075322,0,5900,0,5900,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2
9,10359355,2,7215187,0,200000,0,200000,263184,1,2,7215187,0,200000,0,200000,200000,2022-12-08,2022-12-15,273099,122000,151099,151129,483,2
10,10425987,2,7253109,0,129300,0,129300,313444,1,2,7253109,0,129300,0,129300,116900,2023-06-15,2023-06-22,120547,0,120547,120547,462,2
12,10517437,2,7569765,0,58400,0,58400,157211,1,2,7569765,0,58400,0,58400,107000,2022-07-15,2022-08-05,171200,114500,56700,67908,476,2


In [63]:
batch_2_zeroization = with_limits_and_one_active_loans_history[with_limits_and_one_active_loans_history['principal_disbursed'].isnull()]

batch_2_zeroization.head()

,national_id,store_number_x_x,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x,loan_mifos_id,store_number_y_x,store_number_x_y,store_number_y_y,final_21_limit_y,final_7_limit_y,final_1_limit_y,max_global_limit_y,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
16,10709802,2,753562,0,200000,200000,200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,10795138,2,7952285,0,28500,33100,33100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,11846686,2,4016077,0,39900,0,39900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,11866998,3,157282,0,75400,0,75400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,13248916,2,7622946,0,1800,1400,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
batch_2_zeroization = batch_2_zeroization[['national_id', 'store_number_y', 'final_21_limit_x', 'final_7_limit_x', 'final_1_limit_x', 'max_global_limit_x']]

In [65]:
batch_2_zeroization.head()

,national_id,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x
16,10709802,753562,0,200000,200000,200000
19,10795138,7952285,0,28500,33100,33100
78,11846686,4016077,0,39900,0,39900
80,11866998,157282,0,75400,0,75400
105,13248916,7622946,0,1800,1400,1800


In [66]:
# batch_2_zeroization.to_excel('batch_2_zeroization.xlsx')

In [67]:
batch_2_zeroization['national_id'].nunique()

17

In [68]:
batch_2_zeroization['store_number_y'].nunique()

17

In [69]:
batch_2_zeroization[['final_21_limit_x', 'final_7_limit_x', 'final_1_limit_x', 'max_global_limit_x']].sum()

final_21_limit_x      492100
final_7_limit_x      1570000
final_1_limit_x       453900
max_global_limit_x   1617800
dtype: float64

In [70]:
with_limits_and_loans

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
92,10134016,4,7002703,0,200000,0,200000,313281,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
104,10147213,2,7075322,0,5900,0,5900,312640,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2
167,10359355,2,7215187,0,200000,0,200000,263184,200000,2022-12-08,2022-12-15,273099,122000,151099,151129,483,2
191,10425987,2,7253109,0,129300,0,129300,313444,116900,2023-06-15,2023-06-22,120547,0,120547,120547,462,2
227,10517437,2,7569765,0,58400,0,58400,157211,107000,2022-07-15,2022-08-05,171200,114500,56700,67908,476,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21914,8823131,2,7853741,200000,200000,0,200000,311251,200000,2023-05-27,2023-06-17,218240,0,218240,NaN,484,2
22090,9555312,2,7519601,0,57000,0,57000,213301,57000,2022-09-18,2022-09-25,92127,27500,64627,64618,475,2
22101,9640254,2,7035157,42800,23400,0,42800,312752,40800,2023-06-11,2023-07-02,44521,0,44521,44521,458,2
22123,9759491,2,918455,0,24100,18100,24100,312691,17600,2023-06-10,2023-06-17,18149,0,18149,18149,463,2


In [71]:
value = group[group > 1]

value.sort_values().head()

national_id
11465171    2
24219713    2
24258453    2
24562333    2
24643340    2
Name: store_number_y, dtype: int64

In [72]:
with_limits_and_loans.head()

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
92,10134016,4,7002703,0,200000,0,200000,313281,82000,2023-06-13,2023-06-20,84558,0,84558,NaN,485,2
104,10147213,2,7075322,0,5900,0,5900,312640,4900,2023-06-10,2023-06-17,5053,0,5053,5053,462,2
167,10359355,2,7215187,0,200000,0,200000,263184,200000,2022-12-08,2022-12-15,273099,122000,151099,151129,483,2
191,10425987,2,7253109,0,129300,0,129300,313444,116900,2023-06-15,2023-06-22,120547,0,120547,120547,462,2
227,10517437,2,7569765,0,58400,0,58400,157211,107000,2022-07-15,2022-08-05,171200,114500,56700,67908,476,2


In [73]:
with_limits_and_more_than_one_active_loans = pd.merge(value.to_frame(), with_limits_and_loans, on='national_id', how='left')

with_limits_and_more_than_one_active_loans.head()

,national_id,store_number_y_x,store_number_x,store_number_y_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
0,11465171,2,4,700061,0,66200,0,66200,313252,70000,2023-06-13,2023-06-20,72184,0,72184,NaN,508,2
1,11465171,2,4,700061,0,66200,0,66200,311603,70000,2023-06-05,2023-06-12,73372,55784,17588,6400,508,2
2,11545554,3,4,597324,0,200000,0,200000,313224,200000,2023-06-13,2023-06-20,206240,0,206240,NaN,519,2
3,11545554,3,4,793913,0,200000,0,200000,313005,200000,2023-06-11,2023-06-18,206240,0,206240,NaN,519,2
4,11545554,3,4,788118,200000,200000,0,200000,312748,156800,2023-06-11,2023-06-18,161692,0,161692,161692,519,2


In [74]:
with_limits_and_more_than_one_active_loans.shape

(102, 18)

In [75]:
with_limits_and_more_than_one_active_loans['national_id'].nunique()

48

In [76]:
with_limits_and_more_than_one_active_loans.head()

,national_id,store_number_y_x,store_number_x,store_number_y_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
0,11465171,2,4,700061,0,66200,0,66200,313252,70000,2023-06-13,2023-06-20,72184,0,72184,NaN,508,2
1,11465171,2,4,700061,0,66200,0,66200,311603,70000,2023-06-05,2023-06-12,73372,55784,17588,6400,508,2
2,11545554,3,4,597324,0,200000,0,200000,313224,200000,2023-06-13,2023-06-20,206240,0,206240,NaN,519,2
3,11545554,3,4,793913,0,200000,0,200000,313005,200000,2023-06-11,2023-06-18,206240,0,206240,NaN,519,2
4,11545554,3,4,788118,200000,200000,0,200000,312748,156800,2023-06-11,2023-06-18,161692,0,161692,161692,519,2


In [77]:
with_limits_and_more_than_one_active_loans.shape

(102, 18)

In [78]:
with_limits_and_more_than_one_active_loans[['principal_disbursed', 'total_outstanding', 'safaricom_loan_balance']].sum()

principal_disbursed      10830200
total_outstanding         7963817
safaricom_loan_balance    5776807
dtype: float64

In [79]:
with_limits_merge = pd.merge(with_limits, with_limits_and_more_than_one_active_loans, left_on=['national_id', 'store_number_y', 'loan_mifos_id'], right_on=['national_id', 'store_number_y_y', 'loan_mifos_id'], how='left')

with_limits_merge.head()

,national_id,store_number_x_x,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x,loan_mifos_id,store_number_y_x,store_number_x_y,store_number_y_y,final_21_limit_y,final_7_limit_y,final_1_limit_y,max_global_limit_y,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
0,0328232,3,7164748,110100,60100,0,110100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10033391,2,834417,187100,102100,0,187100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1006543,2,201561,0,76700,0,76700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10134016,4,7002703,0,200000,0,200000,313281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10147213,2,7075322,0,5900,0,5900,312640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
with_limits_merge['store_number_y'].value_counts()

308492     3
798929     2
521670     2
830031     2
514693     2
          ..
940856     1
7609445    1
7088108    1
523548     1
505537     1
Name: store_number_y, Length: 863, dtype: int64

In [81]:
with_limits_and_more_than_one_active_loans[with_limits_and_more_than_one_active_loans['store_number_y_y'] == '308492']

,national_id,store_number_y_x,store_number_x,store_number_y_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
74,27296863,3,3,308492,200000,127100,0,200000,47530,200000,2022-03-19,2022-04-09,227071,215590,11482,NaN,445,2
75,27296863,3,3,308492,200000,127100,0,200000,22878,200000,2022-02-05,2022-02-26,229129,217400,11729,NaN,445,2
76,27296863,3,3,308492,200000,127100,0,200000,311723,45000,2023-06-03,2023-06-10,47286,33159,14127,14720,465,2


In [82]:
with_limits[with_limits['store_number_y'] == '308492']

,national_id,store_number_x,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit,loan_mifos_id
11343,27296863,3,308492,200000,127100,0,200000,47530
11344,27296863,3,308492,200000,127100,0,200000,22878
11345,27296863,3,308492,200000,127100,0,200000,311723


In [83]:
with_limits_merge[with_limits_merge['store_number_y'] == '308492']

,national_id,store_number_x_x,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x,loan_mifos_id,store_number_y_x,store_number_x_y,store_number_y_y,final_21_limit_y,final_7_limit_y,final_1_limit_y,max_global_limit_y,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
595,27296863,3,308492,200000,127100,0,200000,47530,3,3,308492,200000,127100,0,200000,200000,2022-03-19,2022-04-09,227071,215590,11482,NaN,445,2
596,27296863,3,308492,200000,127100,0,200000,22878,3,3,308492,200000,127100,0,200000,200000,2022-02-05,2022-02-26,229129,217400,11729,NaN,445,2
597,27296863,3,308492,200000,127100,0,200000,311723,3,3,308492,200000,127100,0,200000,45000,2023-06-03,2023-06-10,47286,33159,14127,14720,465,2


In [84]:
with_limits_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 894 entries, 0 to 893
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   national_id               894 non-null    object 
 1   store_number_x_x          894 non-null    int64  
 2   store_number_y            894 non-null    object 
 3   final_21_limit_x          894 non-null    int64  
 4   final_7_limit_x           894 non-null    int64  
 5   final_1_limit_x           894 non-null    int64  
 6   max_global_limit_x        894 non-null    float64
 7   loan_mifos_id             342 non-null    float64
 8   store_number_y_x          102 non-null    float64
 9   store_number_x_y          102 non-null    float64
 10  store_number_y_y          102 non-null    object 
 11  final_21_limit_y          102 non-null    float64
 12  final_7_limit_y           102 non-null    float64
 13  final_1_limit_y           102 non-null    float64
 14  max_global

In [85]:
with_limits_merge.shape

(894, 24)

In [86]:
with_limits_and_more_than_one_active_loans_history = with_limits_merge[with_limits_merge.national_id.isin(with_limits_and_more_than_one_active_loans.national_id)]

with_limits_and_more_than_one_active_loans_history.head()

,national_id,store_number_x_x,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x,loan_mifos_id,store_number_y_x,store_number_x_y,store_number_y_y,final_21_limit_y,final_7_limit_y,final_1_limit_y,max_global_limit_y,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
63,11465171,4,700061,0,66200,0,66200,313252,2,4,700061,0,66200,0,66200,70000,2023-06-13,2023-06-20,72184,0,72184,NaN,508,2
64,11465171,4,700061,0,66200,0,66200,311603,2,4,700061,0,66200,0,66200,70000,2023-06-05,2023-06-12,73372,55784,17588,6400,508,2
65,11545554,4,597324,0,200000,0,200000,313224,3,4,597324,0,200000,0,200000,200000,2023-06-13,2023-06-20,206240,0,206240,NaN,519,2
66,11545554,4,793913,0,200000,0,200000,313005,3,4,793913,0,200000,0,200000,200000,2023-06-11,2023-06-18,206240,0,206240,NaN,519,2
67,11545554,4,788118,200000,200000,0,200000,312748,3,4,788118,200000,200000,0,200000,156800,2023-06-11,2023-06-18,161692,0,161692,161692,519,2


In [87]:
with_limits_and_more_than_one_active_loans_history.shape

(107, 24)

In [88]:
batch_3_zeroization = with_limits_and_more_than_one_active_loans_history[with_limits_and_more_than_one_active_loans_history['principal_disbursed'].isnull()]

batch_3_zeroization.head()

,national_id,store_number_x_x,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x,loan_mifos_id,store_number_y_x,store_number_x_y,store_number_y_y,final_21_limit_y,final_7_limit_y,final_1_limit_y,max_global_limit_y,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,total_repayment,total_outstanding,safaricom_loan_balance,src_crdt_score,bloom_version
371,23192831,10,7198135,129300,71100,0,129300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,25758845,5,529095,0,200000,0,200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,26187976,4,7125625,0,200000,200000,200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
554,26242707,6,7145846,0,45800,0,45800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
845,5961828,3,652268,45800,25200,0,45800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
with_limits_and_more_than_one_active_loans_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 63 to 858
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   national_id               107 non-null    object 
 1   store_number_x_x          107 non-null    int64  
 2   store_number_y            107 non-null    object 
 3   final_21_limit_x          107 non-null    int64  
 4   final_7_limit_x           107 non-null    int64  
 5   final_1_limit_x           107 non-null    int64  
 6   max_global_limit_x        107 non-null    float64
 7   loan_mifos_id             102 non-null    float64
 8   store_number_y_x          102 non-null    float64
 9   store_number_x_y          102 non-null    float64
 10  store_number_y_y          102 non-null    object 
 11  final_21_limit_y          102 non-null    float64
 12  final_7_limit_y           102 non-null    float64
 13  final_1_limit_y           102 non-null    float64
 14  max_globa

In [90]:
with_limits_and_more_than_one_active_loans_history.shape

(107, 24)

In [91]:
with_limits_and_more_than_one_active_loans.shape

(102, 18)

In [92]:
batch_3_zeroization = batch_3_zeroization[['national_id', 'store_number_y', 'final_21_limit_x', 'final_7_limit_x', 'final_1_limit_x', 'max_global_limit_x']]

In [93]:
batch_3_zeroization.shape

(5, 6)

In [94]:
batch_3_zeroization['national_id'].nunique()

5

In [95]:
batch_3_zeroization['store_number_y'].nunique()

5

In [96]:
batch_3_zeroization[['final_21_limit_x', 'final_7_limit_x', 'final_1_limit_x', 'max_global_limit_x']].sum()

final_21_limit_x     175100
final_7_limit_x      542100
final_1_limit_x      200000
max_global_limit_x   620900
dtype: float64

In [97]:
# batch_3_zeroization.to_excel('batch_3_zeroization.xlsx')

In [98]:
batch_3_zeroization.head(2)

,national_id,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x
371,23192831,7198135,129300,71100,0,129300
530,25758845,529095,0,200000,0,200000


In [99]:
batch_2_zeroization.head(2)

,national_id,store_number_y,final_21_limit_x,final_7_limit_x,final_1_limit_x,max_global_limit_x
16,10709802,753562,0,200000,200000,200000
19,10795138,7952285,0,28500,33100,33100


In [100]:
batch_1_zeroization.head(2)

,national_id,store_number_y,final_21_limit,final_7_limit,final_1_limit,max_global_limit
465,10967552,829542,0,42400,0,42400
473,10977961,637895,71100,38800,0,71100


In [101]:
batch_1_zeroization = batch_1_zeroization[['national_id', 'store_number_y']]
batch_2_zeroization = batch_2_zeroization[['national_id', 'store_number_y']]
batch_3_zeroization = batch_3_zeroization[['national_id', 'store_number_y']]

zeroization = pd.concat([batch_1_zeroization, batch_2_zeroization, batch_3_zeroization])

zeroization.shape

(63, 2)

In [102]:
zeroization['final_21_limit'] = 0
zeroization['final_7_limit'] = 0
zeroization['final_1_limit'] = 0

zeroization.head()

,national_id,store_number_y,final_21_limit,final_7_limit,final_1_limit
465,10967552,829542,0,0,0
473,10977961,637895,0,0,0
516,11036834,7019742,0,0,0
766,11336526,928643,0,0,0
943,11613645,673772,0,0,0


In [103]:
zeroization = zeroization.rename(columns={'store_number_y':'store_number'})

zeroization.head()

,national_id,store_number,final_21_limit,final_7_limit,final_1_limit
465,10967552,829542,0,0,0
473,10977961,637895,0,0,0
516,11036834,7019742,0,0,0
766,11336526,928643,0,0,0
943,11613645,673772,0,0,0


In [104]:
zeroization.shape

(63, 5)

In [105]:
zeroization['store_number'].nunique()

63

In [106]:
scoring_results = df.copy()

In [107]:
final_21_limit = np.where(scoring_results.store_number.isin(zeroization.store_number), 0, scoring_results.final_21_limit)
final_7_limit = np.where(scoring_results.store_number.isin(zeroization.store_number), 0, scoring_results.final_7_limit)
final_1_limit = np.where(scoring_results.store_number.isin(zeroization.store_number), 0, scoring_results.final_1_limit)

In [108]:
# scoring_results_copy = scoring_results.copy()
scoring_results_copy = df.copy()
scoring_results_copy.final_21_limit = final_21_limit
scoring_results_copy.final_7_limit = final_7_limit
scoring_results_copy.final_1_limit = final_1_limit
scoring_results_copy.loc[scoring_results_copy.store_number.isin(zeroization.store_number), ['final_21_limit', 'final_7_limit', 'final_1_limit']].sum()

final_21_limit    0
final_7_limit     0
final_1_limit     0
dtype: int64

In [109]:
scoring_results_copy.head()

,previous_is_iprs_validated,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,mobile_number,is_location_mapped,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,any_bloom2_1day,total_outstanding_sum,safaricom_loan_balance_sum,loan_balance,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,repayments_by_dd_vs_principal_mean,due_date_rm_ge_rm_add_back_old,due_date_rm_ge_rm_1d_old,due_date_100_check,repayments_by_erd_vs_principal_mean,rllvr_date_rm_ge_rm_add_back_old,rllvr_date_102_check,never_borrowed_flag_old,repayments_by_dd_vs_total_expected_repayment_by_dd_mean,due_date_rm_ge_rm_add_back_new,hurdle_rate_by_due_date_mean,hurdle_rate_by_end_rollover_date_mean,hurdle_rate_by_dpd30_mean,repayments_by_erd_vs_total_expected_repayment_by_erd_mean,rllvr_date_rm_ge_rm_add_back_new,rllvr_date_rm_ge_rm_1d_new,never_borrowed_flag_new,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,due_date_rm_ge_rm_1d,never_borrowed_flag,lrr_update_flag,reinstatement_reason_x,rein_7_limit,idm_recommendation,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,l21_day_graduation_flag_new_limits,21_day_graduation_flag,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit,updated_new_7_limit,updated_new_1_limit,zeroize_final_1_limit,ftd_update_flag,reinstatement_reason_y,model_630_max_global_limit,rein_update_flag,reinstatement_reason,total_max_global_limit,hurdle_rate_dd,hurdle_rate_erd,hurdle_rate_dpd30,rllvr_date_rm_ge_rm_never_borrowed,consolidated_repayments_by_dd,consolidated_repayments_by_erd,consolidated_repayments_by_dpd30,max_global_limit,previous_max_global_limit,max_limit_diff,model_version,created_at,rules_summary_narration,limit_reason,record_added_to_warehouse_on_timestamp
0,None,7356130,2023-05-14,2023-06-12,30,1355574,30,1,relax_rules,0,Yes,1,24782075,True,254724546505,True,3000,254724546505,1,600,7,52000,52000,2023-03-29,2023-04-05,2023-04-04,2023-04-05,2023-04-08,2023-07-07,-1,2,closed_early_repayment,461,52000,2023-03-29,1,1,75,1,-1,1,1,7,1,1,52000,52000,0,52000,-4,False,0,0,0,0,2423200,4675500,0,200000,200000,0,151900,151900,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,None,0,Approve,1,0,0,0,1,1,1,Band 7,Band 2,0,0,0,1,1,1,1,0,0,0,338894,203336,203336,338894,203336,203336,200000,200000,0,153000,pass,pass,200000,200000,0,200000,200000,0,0,200000,200000,200000,200000,0,0,1,1,200000,

In [110]:
scoring_results_copy[['final_21_limit', 'final_7_limit', 'final_1_limit']].sum()

final_21_limit    108626500
final_7_limit     287268300
final_1_limit      57274100
dtype: int64

In [111]:
scoring_results_copy['model_version'] = scoring_results_copy['model_version'].replace(['2023-013[2023-06-14, 2023-06-15]'],'2023-013[2023-06-14, 2023-06-16]')

scoring_results_copy.head()

,previous_is_iprs_validated,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,mobile_number,is_location_mapped,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,any_bloom2_1day,total_outstanding_sum,safaricom_loan_balance_sum,loan_balance,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,repayments_by_dd_vs_principal_mean,due_date_rm_ge_rm_add_back_old,due_date_rm_ge_rm_1d_old,due_date_100_check,repayments_by_erd_vs_principal_mean,rllvr_date_rm_ge_rm_add_back_old,rllvr_date_102_check,never_borrowed_flag_old,repayments_by_dd_vs_total_expected_repayment_by_dd_mean,due_date_rm_ge_rm_add_back_new,hurdle_rate_by_due_date_mean,hurdle_rate_by_end_rollover_date_mean,hurdle_rate_by_dpd30_mean,repayments_by_erd_vs_total_expected_repayment_by_erd_mean,rllvr_date_rm_ge_rm_add_back_new,rllvr_date_rm_ge_rm_1d_new,never_borrowed_flag_new,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,due_date_rm_ge_rm_1d,never_borrowed_flag,lrr_update_flag,reinstatement_reason_x,rein_7_limit,idm_recommendation,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,l21_day_graduation_flag_new_limits,21_day_graduation_flag,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit,updated_new_7_limit,updated_new_1_limit,zeroize_final_1_limit,ftd_update_flag,reinstatement_reason_y,model_630_max_global_limit,rein_update_flag,reinstatement_reason,total_max_global_limit,hurdle_rate_dd,hurdle_rate_erd,hurdle_rate_dpd30,rllvr_date_rm_ge_rm_never_borrowed,consolidated_repayments_by_dd,consolidated_repayments_by_erd,consolidated_repayments_by_dpd30,max_global_limit,previous_max_global_limit,max_limit_diff,model_version,created_at,rules_summary_narration,limit_reason,record_added_to_warehouse_on_timestamp
0,None,7356130,2023-05-14,2023-06-12,30,1355574,30,1,relax_rules,0,Yes,1,24782075,True,254724546505,True,3000,254724546505,1,600,7,52000,52000,2023-03-29,2023-04-05,2023-04-04,2023-04-05,2023-04-08,2023-07-07,-1,2,closed_early_repayment,461,52000,2023-03-29,1,1,75,1,-1,1,1,7,1,1,52000,52000,0,52000,-4,False,0,0,0,0,2423200,4675500,0,200000,200000,0,151900,151900,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,None,0,Approve,1,0,0,0,1,1,1,Band 7,Band 2,0,0,0,1,1,1,1,0,0,0,338894,203336,203336,338894,203336,203336,200000,200000,0,153000,pass,pass,200000,200000,0,200000,200000,0,0,200000,200000,200000,200000,0,0,1,1,200000,

In [112]:
scoring_results_copy.to_excel('scoring_results_copy_20230619.xlsx')

In [114]:
scoring_results_copy.to_parquet('5.0_FM_bloom_scored_limits_risk_review.parquet')

In [113]:
# scoring_results = df[["store_number", "national_id", "final_21_limit", "final_7_limit", "final_1_limit", "idm_recommendation", "limit_factor_21", "limit_factor_7", "limit_factor_1", "model_version", "blacklist_flag", "created_at", "limit_reason"]]

# scoring_results.head()